In [17]:
dataset = read.delim("../input/restaurant-reviews/Restaurant_Reviews.tsv", quote = "", stringsAsFactors = FALSE)

### Limpieza de datos

In [18]:
#install.packages("tm")ç
#install.packages("SnowballC") # Instalación para qu elas stopwords funcionesnn
library(tm)
library(SnowballC)

In [19]:
corpus = VCorpus(VectorSource(dataset$Liked)) # Create corpus
corpus = tm_map(corpus, content_transformer(tolower)) # Transform to lower case
# as.character(corpus[[1]])
corpus = tm_map(corpus, removeNumbers) # Delete number
corpus = tm_map(corpus, removePunctuation) # Delete puntuation
corpus = tm_map(corpus, removeWords, stopwords(kind = "en")) # Eliminan las stopwords (Palabras sin relevancia)
corpus = tm_map(corpus, stemDocument) # Transforma una palabra derivada en la mas simple loving = love
corpus = tm_map(corpus, stripWhitespace)

## Create model Bag of words


In [20]:
dtm = DocumentTermMatrix(corpus) # Creación de matriz dispersa
dtm = removeSparseTerms(dtm, 0.99) # Eliminar palabras que solo salen una vez

In [21]:
dataset$Liked = factor(dataset$Liked, levels = c(0,1))

In [22]:
library(caTools)
set.seed(123)
split <- sample.split(dataset$Liked, SplitRatio = 0.8)

train_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

# Creación del modelo random forest
library(randomForest)
classifier = randomForest(x = train_set[,-692],
                          y = train_set$Liked,
                          ntree = 10)


y_pred = predict(classifier, newdata = test_set[,692]) # Pasa de mostrate probabilidad a ponerte el mas probeble que
# Crear matriz de confusión

cm <- table(test_set[,3], y_pred)